In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os

# -------------------- 1) Detectar y leer anomalos --------------------
base = "anomalos"
file_path = None
for cand in (f"{base}.parquet", f"{base}.csv"):
    if os.path.exists(cand):
        file_path = cand
        break

if not file_path:
    raise FileNotFoundError("No se encontró anomalos.parquet ni anomalos.csv en el directorio actual.")

if file_path.endswith(".parquet"):
    df = pd.read_parquet(file_path)  # requiere que tu entorno tenga soporte parquet (pyarrow/fastparquet)
else:
    df = pd.read_csv(file_path)

# -------------------- 2) Orden de columnas + formato fechas --------------------
cols_wanted = ["cliente","razon_social","nombre_fantasia","telefono",
               "pedido","f_creacion","f_entrega","material","cajas"]
cols = [c for c in cols_wanted if c in df.columns]
df = df[cols].copy()

for c in ("f_creacion","f_entrega"):
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce").dt.date.astype(str)

# -------------------- 3) Utilidades: wrap de texto y anchos --------------------
def wrap_text(value, width):
    """Envuelve texto sin textwrap: corta por espacios hasta 'width' y si no hay espacio, corta en width."""
    if pd.isna(value):
        return ""
    s = str(value).strip()
    if width <= 0:
        return s
    out_lines = []
    while len(s) > width:
        cut = s.rfind(" ", 0, width + 1)
        if cut == -1 or cut < int(width * 0.5):  # si no hay espacio razonable, corta duro
            cut = width
        out_lines.append(s[:cut].rstrip())
        s = s[cut:].lstrip()
    out_lines.append(s)
    return "\n".join(out_lines)

# ancho objetivo (caracteres) por columna — ajusta si te queda muy ancho/angosto
char_width_hint = {
    "cliente": 12,
    "razon_social": 36,
    "nombre_fantasia": 24,
    "telefono": 14,
    "pedido": 14,
    "f_creacion": 12,
    "f_entrega": 12,
    "material": 10,
    "cajas": 8,
}

# Calcular necesidad de ancho (en “caracteres”) por columna según datos
need = []
for c in cols:
    # máximo entre el largo de los datos y el nombre de la columna,
    # acotado por el hint (para que envuelva si es más largo)
    max_len = max([len(str(x)) for x in df[c].astype(str).tolist()] + [len(c)])
    # limitamos el “need” a no más del doble del hint, para que no rompa el layout
    hint = char_width_hint.get(c, 14)
    need.append(min(max_len, hint * 2))

total_need = float(sum(need)) if need else 1.0
w_frac = [n / total_need for n in need]  # fracciones para colWidths en matplotlib.table

# -------------------- 4) Preparar DataFrame envuelto --------------------
df_wrapped = df.copy()
for c, width_chars in zip(cols, [char_width_hint.get(c, 14) for c in cols]):
    df_wrapped[c] = df_wrapped[c].map(lambda v: wrap_text(v, width_chars))

# -------------------- 5) Parámetros de PDF y paginación --------------------
FIG_W, FIG_H = 11.69, 8.27  # A4 landscape en pulgadas
HEADER_COLOR = (0.94, 0.96, 0.99)
FONT_SIZE = 8
HEADER_FONT_SIZE = 9
ROW_HEIGHT = 0.045  # ajusta para más/menos filas por página
rows_per_page = max(10, int(0.80 / ROW_HEIGHT))  # ~80% del alto util

out_pdf = "reporte_con_estilo.pdf"
total_rows = len(df_wrapped)
pages = max(1, (total_rows + rows_per_page - 1) // rows_per_page)

with PdfPages(out_pdf) as pdf:
    for p in range(pages):
        start = p * rows_per_page
        end = min((p + 1) * rows_per_page, total_rows)
        chunk = df_wrapped.iloc[start:end]

        fig, ax = plt.subplots(figsize=(FIG_W, FIG_H))
        ax.axis("off")
        ax.set_title("Reporte de Anómalos", loc="left", fontsize=11, pad=10)

        # Encabezado + filas
        table_data = [cols] + chunk.values.tolist()

        # La tabla usa colWidths como fracciones del ancho del eje
        tab = ax.table(
            cellText=table_data,
            colLabels=None,
            colWidths=w_frac,
            loc="upper left",
            cellLoc="left"
        )
        tab.auto_set_font_size(False)
        tab.set_fontsize(FONT_SIZE)

        # Encabezado (fila 0)
        for j, col_name in enumerate(cols):
            cell = tab[(0, j)]
            cell.set_text_props(text=col_name, fontsize=HEADER_FONT_SIZE, fontweight="bold")
            cell.set_facecolor(HEADER_COLOR)

        # Bordes, altura y padding
        n_rows = len(table_data)
        for i in range(n_rows):
            for j in range(len(cols)):
                cell = tab[(i, j)]
                cell.set_height(ROW_HEIGHT)
                cell.set_edgecolor("black")
                cell.set_linewidth(0.35)
                # padding sencillo
                try:
                    cell.PAD = 0.01
                except Exception:
                    pass

        # Posicionar tabla (un poco debajo del título)
        # [x0, y0, width, height] en coordenadas del eje (0..1)
        table_height = ROW_HEIGHT * n_rows
        y_top = 0.92
        tab._position = [0.02, max(0.08, y_top - table_height), 0.96, table_height]

        # Footer con paginación
        ax.text(0.99, 0.02, f"Página {p+1}/{pages}", ha="right", va="bottom", fontsize=8, transform=ax.transAxes)

        pdf.savefig(fig, bbox_inches="tight")
        plt.close(fig)

print(f"✅ PDF generado: {out_pdf}")


✅ PDF generado: reporte_con_estilo.pdf
